In [4]:
!pip install qiskit qiskit_aer
!pip install qiskit_ibm_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.

In [ ]:
import qiskit
print(qiskit.__version__)

1.3.1


In [ ]:
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

In [ ]:
import hmac
import hashlib


def hmac_prf(secret_key, input_data):
    """
    Generate HMAC using SHA-256 and return the binary output (256 bits).

    Args:
    - secret_key (str): Shared secret key (K).
    - input_data (str): Previous value and identifier for PRF.

    Returns:
    - str: 256-bit binary output from the HMAC function.
    """
    hmac_output = hmac.new(
        secret_key.encode(), input_data.encode(), hashlib.sha256
    ).hexdigest()
    return bin(int(hmac_output, 16))[2:].zfill(256)[:28]


def generate_new_key(K, seed, round_number):
    """
    Generate a new key using HMAC and expand using the previous key (T1, T2, T3, ...).

    Args:
    - K (str): Shared secret key (K).
    - seed (str): A unique seed (e.g., for each QKD round).
    - round_number (int): The round number, used to create unique identifiers (0x01, 0x02, ...).

    Returns:
    - str: A new key derived from K using HMAC-based PRF.
    """
    # Construct identifier (e.g., 0x01, 0x02, ...)
    identifier = f"0x{round_number:02X}"

    # Combine the previous output with the seed and identifier for HMAC input
    input_data = f"{K}|{seed}|{identifier}"

    # Get the next key (T1, T2, ...)
    return hmac_prf(K, input_data)

In [ ]:
# Shared Key (K) and Initial Seed (S)
shared_key = "shared_secret_key"  # Shared secret key (K)
seed = "shared_seed"  # Shared seed (S)

# Generate multiple keys (T1, T2, T3, ...) based on shared key (K) and seed (S)
rounds = 5 # Example: number of rounds you want to generate keys for

keys = []
for round_number in range(1, rounds + 1):
    new_key = generate_new_key(shared_key, seed, round_number)
    keys.append(new_key)
    print(f"T{round_number} = {new_key}")  # Display each generated key

T1 = 1011001000111101111111000111
T2 = 0100100100100101100110101010
T3 = 1000010010011001001001001100
T4 = 0110010110001000101000111011
T5 = 0100110100110011000101011000


In [ ]:
keys[0]

'1011001000111101111111000111'

In [ ]:
alice_bases = keys[0]
bob_bases = keys[0]

# Real Hardware

In [5]:
token= "90c11c3427d32b9170daa73f5fffe26dc68634b8b6f0d8e5a83082726bd42c2d384e16c5ea3f3ed03f5cb71156de8fb4ba9b3880f9f990c6ed2d275f315de5d3"
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

QiskitRuntimeService.save_account(
  token=token,
  channel="ibm_quantum",
  overwrite=True
)


In [6]:
service = QiskitRuntimeService()
n_qubits = 5
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=n_qubits)
print("Selected Backend", backend)

Selected Backend <IBMBackend('ibm_brisbane')>


In [ ]:
# Function to generate a random binary string using quantum circuits
def RandomStringIBM(str_len):
    op_str = ''  # Initialize an empty output string
    num_qbits = 10  # Define number of qubits

    # Calculate the number of chunks needed
    num_chunks = math.ceil(str_len / num_qbits)
    for _ in range(num_chunks):
        # Create a quantum register and a classical register
        q = QuantumRegister(num_qbits)
        c = ClassicalRegister(num_qbits)
        QC = QuantumCircuit(q, c)

        # Apply Hadamard gates to all qubits to create superposition
        for i in range(num_qbits):
            QC.h(q[i])
        QC.measure(q, c)
        pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
        isa_circuit = pm.run(QC)

        sampler = Sampler(backend)
        job = sampler.run([isa_circuit])
        result = job.result()


In [ ]:
len_message = 10
alice_key = RandomStringIBM(len_message)

In [7]:
from qiskit_ibm_runtime import QiskitRuntimeService
job = service.job('cy3rmfscw2k0008jmtrg')
result = job.result()

In [40]:
data = result[0].data
bitarray = next(iter(data.values()))
counts = bitarray.get_counts()

In [46]:
# Calculate the sum of counts
total_shots = sum(counts.values())
print("Total measurements (sum):", total_shots)

Total measurements (sum): 4096


In [43]:
first_bitstring = next(iter(counts.keys()))
print(first_bitstring)

1010111110


In [44]:
print(counts)

{'1010111110': 4, '1000111010': 7, '1010100001': 6, '0000101000': 9, '1010001100': 4, '0101110110': 4, '0010100100': 12, '1001101000': 5, '0100010110': 3, '1101101111': 5, '1111101011': 4, '0010100001': 10, '1011000001': 2, '0011100111': 3, '1001001110': 4, '0010000000': 8, '1010100010': 7, '1111001111': 9, '1100101010': 4, '0110100101': 9, '0101000010': 4, '0110111111': 6, '0001101101': 4, '0100001110': 6, '0010100110': 6, '0001001011': 7, '0011000001': 2, '1100101000': 2, '1000111111': 8, '1100111111': 5, '0011011110': 3, '0110101110': 8, '0111011011': 3, '1001101100': 6, '1011110111': 2, '0100001011': 6, '1111101111': 5, '0010100111': 8, '1100001111': 8, '0000000111': 4, '0100110001': 3, '0110101010': 4, '0110001010': 4, '1001001100': 6, '1011111101': 4, '1010101000': 5, '0100101001': 5, '0001111011': 4, '1011111110': 2, '0101111110': 8, '0100000111': 3, '1111000011': 5, '1111111100': 6, '1010100110': 3, '0000001111': 7, '0011101011': 8, '0011100010': 7, '0001110111': 4, '0001111001

# Simulation

In [ ]:
# Function to generate a random binary string using quantum circuits
def RandomString(str_len):
    op_str = ''  # Initialize an empty output string
    num_qbits = 10  # Define number of qubits

    # Calculate the number of chunks needed
    num_chunks = math.ceil(str_len / num_qbits)
    for _ in range(num_chunks):
        # Create a quantum register and a classical register
        q = QuantumRegister(num_qbits)
        c = ClassicalRegister(num_qbits)
        QC = QuantumCircuit(q, c)

        # Apply Hadamard gates to all qubits to create superposition
        for i in range(num_qbits):
            QC.h(q[i])
        QC.measure(q, c)

        simulator = AerSimulator()
        # Measure the qubits
        circ = transpile(QC, simulator)
        result = simulator.run(circ, shots=1, memory=True).result()
        memory = result.get_memory(circ)
        op_str += memory[0]  # Append the measured result to op_str

    return op_str[:str_len]  # Return the full random string

In [ ]:
len_message = 28
alice_key = RandomString(len_message)

In [ ]:
# Quantum Circuit for encoding
q = QuantumRegister(len_message)
c = ClassicalRegister(len_message)
qc = QuantumCircuit(q, c)

# Encode qubits based on Alice's key and basis
for i in range(len_message):
    if alice_key[i] == '1':
        qc.x(q[i])  # Flip the qubit to 1 if key is 1
    if alice_bases[i] == '1':

        qc.h(q[i])  # Apply Hadamard gate if basis is 1
qc.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=28, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(28, 'q4'), 0), Qubit(QuantumRegister(28, 'q4'), 1), Qubit(QuantumRegister(28, 'q4'), 2), Qubit(QuantumRegister(28, 'q4'), 3), Qubit(QuantumRegister(28, 'q4'), 4), Qubit(QuantumRegister(28, 'q4'), 5), Qubit(QuantumRegister(28, 'q4'), 6), Qubit(QuantumRegister(28, 'q4'), 7), Qubit(QuantumRegister(28, 'q4'), 8), Qubit(QuantumRegister(28, 'q4'), 9), Qubit(QuantumRegister(28, 'q4'), 10), Qubit(QuantumRegister(28, 'q4'), 11), Qubit(QuantumRegister(28, 'q4'), 12), Qubit(QuantumRegister(28, 'q4'), 13), Qubit(QuantumRegister(28, 'q4'), 14), Qubit(QuantumRegister(28, 'q4'), 15), Qubit(QuantumRegister(28, 'q4'), 16), Qubit(QuantumRegister(28, 'q4'), 17), Qubit(QuantumRegister(28, 'q4'), 18), Qubit(QuantumRegister(28, 'q4'), 19), Qubit(QuantumRegister(28, 'q4'), 20), Qubit(QuantumRegister(28, 'q4'), 21), Qubit(QuantumRegister(28, 'q4'), 22), Qubit(QuantumRegister(28, 'q4'

In [ ]:
qc.draw()

┌───┐┌───┐ ░ 
 q4_0: ┤ X ├┤ H ├─░─
       ├───┤└───┘ ░ 
 q4_1: ┤ X ├──────░─
       ├───┤┌───┐ ░ 
 q4_2: ┤ X ├┤ H ├─░─
       ├───┤├───┤ ░ 
 q4_3: ┤ X ├┤ H ├─░─
       ├───┤└───┘ ░ 
 q4_4: ┤ X ├──────░─
       └───┘      ░ 
 q4_5: ───────────░─
       ┌───┐┌───┐ ░ 
 q4_6: ┤ X ├┤ H ├─░─
       ├───┤└───┘ ░ 
 q4_7: ┤ X ├──────░─
       └───┘      ░ 
 q4_8: ───────────░─
                  ░ 
 q4_9: ───────────░─
       ┌───┐      ░ 
q4_10: ┤ H ├──────░─
       ├───┤┌───┐ ░ 
q4_11: ┤ X ├┤ H ├─░─
       ├───┤└───┘ ░ 
q4_12: ┤ H ├──────░─
       ├───┤      ░ 
q4_13: ┤ H ├──────░─
       ├───┤      ░ 
q4_14: ┤ X ├──────░─
       ├───┤      ░ 
q4_15: ┤ H ├──────░─
       ├───┤┌───┐ ░ 
q4_16: ┤ X ├┤ H ├─░─
       ├───┤├───┤ ░ 
q4_17: ┤ X ├┤ H ├─░─
       ├───┤├───┤ ░ 
q4_18: ┤ X ├┤ H ├─░─
       ├───┤└───┘ ░ 
q4_19: ┤ H ├──────░─
       ├───┤      ░ 
q4_20: ┤ H ├──────░─
       ├───┤      ░ 
q4_21: ┤ H ├──────░─
       ├───┤      ░ 
q4_22: ┤ X ├──────░─
       ├───┤      ░ 
q4_23: ┤ X ├──────░─
       ├───┤      ░ 
q4_24: ┤ X ├──────░─
       ├───┤┌───┐ ░ 
q4_25: ┤ X ├┤ H ├─░─
       ├───┤└───┘ ░ 
q4_26: ┤ H ├──────░─
       ├───┤      ░ 
q4_27: ┤ H ├──────░─
       └───┘      ░ 
c3: 28/═════════════

In [ ]:
# Step 3: Bob measures the qubits
for i in range(len_message):
    if bob_bases[i] == '1':
        qc.h(q[i])  # Bob applies Hadamard if measuring in Hadamard basis

# Add measurements
qc.measure(q, c)

In [ ]:
# Run the circuit
simulator = AerSimulator()
circ = transpile(qc, simulator)
result = simulator.run(circ, shots=1, memory=True).result()
bob_result = result.get_memory(circ)
print('Bob Measurement Result:', bob_result[0])

# Display matching bits for shared key
shared_key = []
for i in range(len_message):
    if alice_bases[i] == bob_bases[i]:  # Only keep the bit if bases match
        shared_key.append(bob_result[0][len_message - i - 1])  # Reverse order in Qiskit

final_shared_key = ''.join(shared_key)
print('Final Shared Key:', final_shared_key)

# Calculate the percentage of key bits retained
matching_bits_count = len(final_shared_key)
percentage_retained = (matching_bits_count / len_message) * 100
print("Percentage of key bits retained: ", percentage_retained, "%")

Bob Measurement Result: 0011110001110100100011011111
Final Shared Key: 1111101100010010111000111100
Percentage of key bits retained:  100.0 %


Job id = 773e84cb-4b67-405e-8383-9bdeed4124e8

In [ ]:
len(final_shared_key)

28

In [ ]:
import random

# Function to convert a string to binary
def text_to_bin(text):
    return ''.join(format(ord(c), '08b') for c in text)

# Function to convert binary back to string
def bin_to_text(binary_string):
    chars = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
    return ''.join(chr(int(char, 2)) for char in chars)

# Function to encrypt a message using One-Time Pad
def encrypt(message, key):
    # Convert the message to binary
    binary_message = text_to_bin(message)

    # Encrypt by XORing the binary message with the key (key length fixed at 28)
    encrypted = ''.join(str(int(binary_message[i]) ^ int(key[i % len(key)])) for i in range(len(binary_message)))
    return encrypted

# Function to decrypt a message using One-Time Pad
def decrypt(ciphertext, key):
    # Decrypt by XORing the ciphertext with the key
    decrypted = ''.join(str(int(ciphertext[i]) ^ int(key[i % len(key)])) for i in range(len(ciphertext)))

    # Convert the decrypted binary string back to text
    return bin_to_text(decrypted)

# Example usage
message = "Helod World"

# Assuming final_shared_key is predefined and has a length of 28

print(f"Key: {final_shared_key}")

# Encrypt the message using One-Time Pad
ciphertext = encrypt(message, final_shared_key)
print(f"Encrypted Message (Binary): {ciphertext}")

# Decrypt the message using the same key
decrypted_message = decrypt(ciphertext, final_shared_key)
print(f"Decrypted Message: {decrypted_message}")


Key: 0111010001110110010100000100
Encrypted Message (Binary): 0011110000010011001111000010100000100011010001010101001100011011000001000011110000100011
Decrypted Message: Helod World
